## 1. 환경 설정

### 1-1. 환경 변수

In [267]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### 1-2. 기본 라이브러리

In [268]:
import re
import json

from textwrap import dedent
from pprint import pprint

import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import uuid

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 30)

## 2. 데이터 설정

타겟 자산명 (직접 작성)

In [269]:
asset_names = ['AA-ETF_EQUITY-XLU']
# asset_names = ['AA-ETF_EQUITY-XLK', 'AA-ETF_EQUITY-XLE', 'AA-ETF_EQUITY-XLF', 'AA-ETF_EQUITY-XLV', 'AA-ETF_EQUITY-XLP', 'AA-ETF_EQUITY-XLI', 'AA-ETF_EQUITY-XLU']
# asset_names = ['AA-ETF_EQUITY-EWY', 'AA-ETF_EQUITY-VWO', 'AA-ETF_EQUITY-EWC', 'AA-ETF_EQUITY-EFA', 'AA-ETF_EQUITY-EWJ', 'AA-ETF_EQUITY-EPP']
# asset_names = ['AA-ETF_EQUITY-ACWV', 'AA-ETF_EQUITY-SUSL', 'AA-ETF_EQUITY-SCHD', 'AA-ETF_EQUITY-VYMI', 'AA-ETF_EQUITY-VTV'] # ['AA-ETF_EQUITY-ACWV.K', 'AA-ETF_EQUITY-SUSL.O', 'AA-ETF_EQUITY-SCHD.K', 'AA-ETF_EQUITY-VYMI.O', 'AA-ETF_EQUITY-VTV']

### 2-1. 자산배분

In [270]:
df_alloc = pd.read_csv('data/alloc_basdt_241217.csv')
df_alloc

,BAS_DT,ASET_GRP,ASET_GRP_WGT
0,2024-11-20,AA-ETF_BOND-BND,0.0004
1,2024-11-20,AA-ETF_BOND-BNDX,0.0011
2,2024-11-20,AA-ETF_EQUITY-ACWV,0.0030
3,2024-11-20,AA-ETF_EQUITY-EFA,0.0005
4,2024-11-20,AA-ETF_EQUITY-EPP,0.0003
...,...,...,...
455,2024-12-17,AA-ETF_EQUITY-XLK,0.4481
456,2024-12-17,AA-ETF_EQUITY-XLP,0.0103
457,2024-12-17,AA-ETF_EQUITY-XLU,0.0145
458,2024-12-17,AA-ETF_EQUITY-XLV,0.0099


### 2-2. 설명력

In [271]:
# 데이터 전처리
df_infc = pd.read_csv('data/infc_basdt_241217.csv')
df_infc['ASET_GRP'] = df_infc['ASET_LEVEL'].str[3:]
df_infc['ASET_LEVEL'] = df_infc['ASET_LEVEL'].str[:2]
df_infc = df_infc[df_infc['ASET_GRP'].isin(asset_names)] # RIC 중복 존재 (기간 통합, L1-L3)

df_infc = df_infc.merge(df_alloc, on=['BAS_DT', 'ASET_GRP'], how='inner')
# df_infc = df_infc[['BAS_DT', 'ASET_GRP', 'ASET_GRP_WGT', 'RIC', 'VAR_NM', 'INFC_DIR']]
df_infc = df_infc.reset_index(drop=True)

# 테이블명 추가
df_var_map = pd.read_csv('data/var_table_mapping.csv')
df_var_map = df_var_map[['RIC', 'TABLE']]
df_var_map = df_var_map.drop_duplicates('RIC') # RIC 중복 제거 (필드만 다른 경우 존재)
df_infc = df_infc.merge(df_var_map, on='RIC', how='inner')

df_infc = df_infc.reset_index(drop=True)
df_infc # 최종

,BAS_DT,XAI_LEVEL_NM,ASET_LEVEL,RIC,VAR_NM,INFC_SCRE,INFC_RNK,INFC_DIR,ASET_GRP,ASET_GRP_WGT,TABLE
0,2024-11-20,자산군,L3,MCU0,London Metal Exchange (LME)-Copper Grade A Cas...,0.32,1,-,AA-ETF_EQUITY-XLU,0.0059,DSC105TH
1,2024-11-20,자산군,L3,EUR2M=,United States Dollar to Euro 2 Month Forward P...,0.19,2,-,AA-ETF_EQUITY-XLU,0.0059,DSC104TH
2,2024-11-20,자산군,L3,XAU=,"Gold, USD FX Composite U United States Dollar ...",0.18,3,-,AA-ETF_EQUITY-XLU,0.0059,DSC105TH
3,2024-11-20,자산군,L3,STXEc1,EUREX-DJ EURO STOXX 50 TRc1,0.15,4,-,AA-ETF_EQUITY-XLU,0.0059,DSC103TH
4,2024-11-20,자산군,L3,.FTSE,FTSE 100,0.14,5,-,AA-ETF_EQUITY-XLU,0.0059,DSC101TH
...,...,...,...,...,...,...,...,...,...,...,...
194,2024-12-17,자산군,L3,ESc1,CME-MINI S&P 500 INDEX TRc1,0.61,5,-,AA-ETF_EQUITY-XLU,0.0145,DSC103TH
195,2024-12-17,자산군,L3,FCc1,CME-Feeder Cattle Composite TRC1,0.56,6,-,AA-ETF_EQUITY-XLU,0.0145,DSC103TH
196,2024-12-17,자산군,L3,JNIc1,OSX-NIKKEI 225 INDEX TRc1,0.54,7,-,AA-ETF_EQUITY-XLU,0.0145,DSC103TH
197,2024-12-17,자산군,L3,LSUc1,LIFFE-WHITE SUGAR TRc1,0.35,15,+,AA-ETF_EQUITY-XLU,0.0145,DSC103TH


### 2-3. 기초 데이터

In [272]:
# 전체 RIC (데이터 존재하는 것들만)
RIC_101 = ['.AORD', '.BVSP', '.FTSE', '.KS11']
RIC_102 = ['BR10YT=RR', 'BR3YT=RR', 'KR3YT=RR']
RIC_103 = ['BZZc1', 'Cc1', 'ESc1', 'FCc1', 'HOc1', 'INDc1', 'JNIc1', 'LCc1', 'LSUc1', 'PAc1', 'PLc1', 'STXEc1']
RIC_104 = ['EUR1M=', 'EUR2M=', 'JPY=']
RIC_105 = ['MCU0', 'XAU=']

In [273]:
# 전체 변수 (데이터 존재하는 것들만)
var_101 = [df_infc[df_infc['RIC'] == RIC_101[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_101))]
var_102 = [df_infc[df_infc['RIC'] == RIC_102[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_102))]
var_103 = [df_infc[df_infc['RIC'] == RIC_103[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_103))]
var_104 = [df_infc[df_infc['RIC'] == RIC_104[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_104))]
var_105 = [df_infc[df_infc['RIC'] == RIC_105[i]]['VAR_NM'].iloc[0] for i in range(len(RIC_105))]

In [274]:
var_dict = {
    'Equity and Volatility Indicators': var_101,
    'Government Bond Yields': var_102,
    'Commodities and Futures': var_103,
    'Currency Exchange Rates': var_104,
    'Market Index': var_105
    }
var_dict

{'Equity and Volatility Indicators': ['Standard and Poors / ASX All Ordinaries Gold Open',
  'BRAZIL BOVESPA',
  'FTSE 100',
  'Korea Stock Exchange Composite (KOSPI)'],
 'Government Bond Yields': ['Refinitiv Brazil Government Benchmark Bid Yield 10 Years',
  'Refinitiv Brazil Government Benchmark Bid Yield 3 Years',
  'Refinitiv S Korea Government Benchmark Bid Yield 3 Years'],
 'Commodities and Futures': ['NYMEX - Brent Crude Oil LST Day TRC1',
  'Chicago Board of Trade (CBOT)-Corn Composite TRC1',
  'CME-MINI S&P 500 INDEX TRc1',
  'CME-Feeder Cattle Composite TRC1',
  'NYM-NY HARBOR ULSD TRc1',
  'BMF-BOVESPA INDEX TRc1',
  'OSX-NIKKEI 225 INDEX TRc1',
  'CME-LIVE CATTLE COMP. TRc1',
  'LIFFE-WHITE SUGAR TRc1',
  'NYM-PALLADIUM TRc1',
  'NYM-PLATINUM TRc1',
  'EUREX-DJ EURO STOXX 50 TRc1'],
 'Currency Exchange Rates': ['United States Dollar to Euro 1 Month Forward Points',
  'United States Dollar to Euro 2 Month Forward Points',
  'Japanese Yen to United States Dollar (Refinitiv)']

In [275]:
var_dict_101 = dict(zip(RIC_101, var_101))
var_dict_102 = dict(zip(RIC_102, var_102))
var_dict_103 = dict(zip(RIC_103, var_103))
var_dict_104 = dict(zip(RIC_104, var_104))
var_dict_105 = dict(zip(RIC_105, var_105))

In [276]:
try:
   df_101 = pd.read_csv(f'data/L3/{asset_names[0].split('-')[-1]}/df_101_basdt_241217.csv')
   df_101['VAR_NM'] = df_101['RIC'].map(var_dict_101)
   df_101 = df_101[['TRADEDATE', 'RIC', 'VAR_NM', 'PI']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_101.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "PI"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_101_data = json.dumps(grouped_data, indent=2)
except:
   table_101_data = ''

In [277]:
try:
   df_102 = pd.read_csv(f'data/L3/{asset_names[0].split('-')[-1]}/df_102_basdt_241217.csv')
   df_102['VAR_NM'] = df_102['RIC'].map(var_dict_102)
   df_102 = df_102[['TRADEDATE', 'RIC', 'VAR_NM', 'RY']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_102.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "RY"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_102_data = json.dumps(grouped_data, indent=2)
except:
   table_102_data = ''

In [278]:
try:
   df_103 = pd.read_csv(f'data/L3/{asset_names[0].split('-')[-1]}/df_103_basdt_241217.csv')
   df_103['VAR_NM'] = df_103['RIC'].map(var_dict_103)
   df_103 = df_103[['TRADEDATE', 'RIC', 'VAR_NM', 'PS']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_103.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "PS"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_103_data = json.dumps(grouped_data, indent=2)
except:
   table_103_data = ''

In [279]:
try:
   df_104 = pd.read_csv(f'data/L3/{asset_names[0].split('-')[-1]}/df_104_basdt_241217.csv')
   df_104['VAR_NM'] = df_104['RIC'].map(var_dict_104)
   df_104 = df_104[['TRADEDATE', 'RIC', 'VAR_NM', 'ER']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_104.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "ER"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_104_data = json.dumps(grouped_data, indent=2)
except:
   table_104_data = ''

In [280]:
try:
   df_105 = pd.read_csv(f'data/L3/{asset_names[0].split('-')[-1]}/df_105_basdt_241217.csv')
   df_105['VAR_NM'] = df_105['RIC'].map(var_dict_105)
   df_105 = df_105[['TRADEDATE', 'RIC', 'VAR_NM', 'P']]

   # Grouping and restructuring the data
   grouped_data = []
   for (ric, var_nm), group in df_105.groupby(["RIC", "VAR_NM"]):
      entries = group[["TRADEDATE", "P"]].to_dict(orient="records")
      grouped_data.append({"RIC": ric, "VAR_NM": var_nm, "entries": entries})

   # Output JSON
   table_105_data = json.dumps(grouped_data, indent=2)
except:
   table_105_data = ''

### 2-4. 모델 설정

In [281]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

llm = ChatOpenAI(model="gpt-4o", temperature=0)

### 2-5. 설명 타겟

In [282]:
AI_model_result = df_infc.drop_duplicates(['BAS_DT', 'ASET_GRP'])[['BAS_DT', 'ASET_GRP', 'ASET_GRP_WGT']].reset_index(drop=True)

# Grouping and restructuring the data
grouped_data = []
for aset_grp, group in AI_model_result.groupby("ASET_GRP"):
    entries = group[["BAS_DT", "ASET_GRP_WGT"]].to_dict(orient="records")
    grouped_data.append({"ASET_GRP": aset_grp, "entries": entries})

# Output JSON
AI_model_result = json.dumps(grouped_data, indent=2)

AI_model_result = llm.invoke(
        [SystemMessage(content="Extract 3 most notable weight changes in the portfolio optimization results below. List them from relatively easy to explain to complex. There should be no overlap in contents.")] + 
        [HumanMessage(content=AI_model_result)]
        )
AI_model_result = AI_model_result.content
print(AI_model_result)

1. **Gradual Increase in Weight (November 20 to December 16, 2024):** The weight of the asset group "AA-ETF_EQUITY-XLU" shows a steady and gradual increase from 0.0059 on November 20 to 0.0101 on December 16. This change is relatively easy to explain as it could be attributed to a consistent positive performance or a strategic decision to increase exposure to this asset group over time.

2. **Significant Jump in Weight (December 16 to December 17, 2024):** There is a notable and sudden increase in the weight from 0.0101 on December 16 to 0.0145 on December 17. This significant jump could be due to a major event or news affecting the asset group, such as a regulatory change, a merger, or a significant market movement that prompted a rapid adjustment in the portfolio.

3. **Overall Trend and Strategic Implications:** The overall trend from November 20 to December 17 shows a consistent increase in the weight of the asset group, culminating in a sharp rise on December 17. This complex chan

### 2-6. 설명력 스코어

In [283]:
XAI_model_result = df_infc[['BAS_DT', 'VAR_NM', 'INFC_SCRE', 'INFC_DIR']].reset_index(drop=True)

# Grouping and restructuring the data
grouped_data = []
for var_nm, group in XAI_model_result.groupby("VAR_NM"):
    entries = group[['BAS_DT', 'INFC_SCRE', 'INFC_DIR']].to_dict(orient="records")
    grouped_data.append({"VAR_NM": var_nm, "entries": entries})

# Output JSON
XAI_model_result = json.dumps(grouped_data, indent=2)

XAI_model_result = llm.invoke(
        [SystemMessage(content=f"Interpret the temporal trend of daily influence score data (Shapley value) of explanatory variables for daily weight determination of {asset_names[0]} in the portfolio. List them in order of Meaning of Correlation (as you understand), Positive Correlations, Negative Correlations, and Other Variables, one paragraph each. Positive and negative scores indicate positive and negative correlations, respectively. Higher absolute value of influence score means higher influence. Preserve important details and include no preamble.")] + 
        [HumanMessage(content=XAI_model_result)]
        )
XAI_model_result = XAI_model_result.content
print(XAI_model_result)

Positive Correlations: The variables with positive correlations include "CME-LIVE CATTLE COMP. TRc1" with a high influence score of 0.93, "BMF-BOVESPA INDEX TRc1" with a score of 0.91, and "LIFFE-WHITE SUGAR TRc1" with a score of 0.35. Other positively correlated variables with moderate influence scores are "NYMEX - Brent Crude Oil LST Day TRC1" at 0.25, "Chicago Board of Trade (CBOT)-Corn Composite TRC1" peaking at 0.19, and "BRAZIL BOVESPA" reaching 0.18. Additionally, "Japanese Yen to United States Dollar (Refinitiv)" and "United States Dollar to Euro 1 Month Forward Points" both show influence scores up to 0.19, while "NYM-NY HARBOR ULSD TRc1" and "Refinitiv Brazil Government Benchmark Bid Yield 10 Years" have lower scores around 0.13. "NYM-PLATINUM TRc1" also shows a positive trend with scores up to 0.20.

Negative Correlations: The variables with negative correlations include "CME-MINI S&P 500 INDEX TRc1" with a score of 0.61, "EUREX-DJ EURO STOXX 50 TRc1" with a score of 0.62, a

## 3. Agent

### 3-1. Analyst 생성

In [284]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from IPython.display import Image, display
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

class Analyst(BaseModel):
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst.")
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    description: str = Field(description="Description of the analyst's focus, concerns, and motives.")

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"

In [285]:
import operator
from typing import List, Annotated
from typing_extensions import TypedDict

class ResearchGraphState(TypedDict):
    human_analyst_feedback: str
    analysts: List[Analyst]
    sections: Annotated[list, operator.add]
    # interpretations: Annotated[list, operator.add]
    content: str

In [286]:
def create_analysts(state: ResearchGraphState):
    """ No-op node for creating analysts """
    pass

In [287]:
def human_feedback(state: ResearchGraphState):
    """ No-op node that should be interrupted on """
    pass

### 3-2. Interview 서브그래프 정의

#### A. Question 생성

In [288]:
import operator
from typing import Annotated
from langgraph.graph import MessagesState

class InterviewState(MessagesState):
    max_num_turns: int
    analyst: Analyst
    context: Annotated[list, operator.add]
    interview: str
    sections: list

In [289]:
num_subquestions = 3
max_num_turns = 3

analysts = [Analyst(name='Emma Thompson', role='Equity Market Analyst', affiliation='Global Equities Research', description='Emma focuses on the dynamics of equity markets, particularly the impact of global indices such as the FTSE 100, KOSPI, and BOVESPA on portfolio optimization. She is concerned with how these indices reflect broader economic trends and investor sentiment, and how they interact with volatility indicators like the ASX All Ordinaries Gold Open.'),
 Analyst(name='Carlos Mendes', role='Fixed Income Strategist', affiliation='Latin America Bond Insights', description='Carlos specializes in government bond yields, with a particular focus on Brazilian and South Korean markets. He analyzes how changes in benchmark yields, such as the 10-year and 3-year Brazilian government bonds, influence investment strategies and risk assessments in portfolio optimization.'),
 Analyst(name='Sophia Lee', role='Commodities Analyst', affiliation='Global Commodities Watch', description='Sophia examines the role of commodities and futures in portfolio optimization, focusing on key indicators like Brent Crude Oil, Corn, and precious metals such as Palladium and Platinum. Her analysis includes understanding how these commodities influence market dynamics and investor behavior.'),
 Analyst(name="Liam O'Connor", role='Currency Market Analyst', affiliation='International Forex Review', description='Liam provides insights into currency exchange rates, particularly the USD to Euro forward points and the JPY to USD exchange rate. He focuses on how these rates affect international trade and investment flows, without engaging in pair trading strategies.'),
 Analyst(name='Isabella Rossi', role='Market Index Specialist', affiliation='Metals and Indices Analysis', description="Isabella's expertise lies in market indices, with a focus on the London Metal Exchange's Copper Grade A and Gold USD FX Composite. She analyzes how these indices serve as economic indicators and their impact on portfolio diversification and risk management.")]

In [290]:
question_instructions = """You are an analyst tasked with interviewing an expert. 

Your goal is to explain the AI model result of portfolio optimization from your professional perspective and data.

It is extremely important to focus on gathering information on high-level themes such as market dynamics, fund flows, and potential risks.
(Caution: Focus on a subset of explanatory variables. DO NOT focus on too many or too few explanatory variables.)

Your target assets:
{asset_names}.

Your explanation target (AI model result):
{AI_model_result}.

Explanatory variables that AI model considered important (Note: Daily data exists for past days.):
{var_dict}.

Reference information to use when asking question (Explainable AI model result):
{XAI_model_result}

Your persona:
{persona}.

Your task:

1. Start interview by introducing yourself.
2. Come up with a challenging and complex first question for the first explanation target.
3. Break this first question into very specific sub-questions. Make {num_subquestions} sub-questions.
  (Caution: From the original question, make sub-questions that can be answered/inferred directly from your knowledge and the past daily data of one or more explanatory variables.
   Select explanatory variables that are closely related to your persona. Sub-questions should not be too trivial or complex. Use exact name for explanatory variables.)
4. Review the answer from the expert and state your overall understanding of previous sub-questions (`## Understanding` header).
   Then, move on to the next explanation target and come up with a new challenging and complex question, saying "Let's move on to the next explanation target!".
  (Note: There are total 3 explanation targets.)
   Also, break the new question into sub-questions (`## Questions` header).
  (Caution: From the original question, make sub-questions that can be answered/inferred directly from your knowledge and the past daily data of one or more explanatory variables.
   Select explanatory variables that are closely related to your persona. Sub-questions should not be too trivial or complex. Use exact name for explanatory variables.)
   Through iteration, boil down to interesting and specific insights.
    - Interesting: Insights that people will find surprising or non-obvious.   
    - Specific: Insights that avoid generalities and include specific figures.
5. To complete the interview, use these criterion:
    - You are very confident about being ready to explain the target result.
    - You have gained enough interesting and specific insights. 
    - There are no more appropriate questions.
   
   Complete the interview like this:
    - Say "I'd like to finish my interview with the expert since I'm ready for impressive explanation!".
    - Briefly state the prepared explanation and how this explanation can be considered an achievement of your goal.

Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:

   ## Understanding
   <Overall understanding>
   ## Questions
   1. [Question] <Follow-up question>
   2. [Sub-questions]
      [Sub-question 1] <First sub-question>
      [Sub-question 2] <Second sub-question>
      [Sub-question 3] <Third sub-question>.

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied."""

def generate_question(state: InterviewState):
    """ Node to generate a question """

    # Get state
    analyst = state["analyst"]
    messages = state["messages"]
    
    # LLM
    system_message = question_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result,
        var_dict=var_dict,
        XAI_model_result=XAI_model_result,
        persona=analyst.persona,
        num_subquestions=num_subquestions
        )
    question = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    # Write messages to state
    return {"messages": [question]}

#### B. Context 추출 & Answer 생성

In [291]:
from langchain_core.messages import get_buffer_string

# Extract context
extract_instructions = """You will be given a conversation between an analyst and an expert.

Your goal is to help the expert answer the analyst's questions by providing context.

Your target assets:
{asset_names}.

Your task:

1. Analyze the full conversation and pay particular attention to the analyst's last message, which contains sub-questions.
2. Extract important context for each sub-question from table (`## Contexts` header).
   Follow these guidelines:
    - Handle ALL sub-questions posed by the analyst at last turn, one by one.
    - When there is no context to extract, DO NOT make up any context and just return "No context for this sub-question!".
3. Include the source (`### Source` header).

Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:

   ## Contexts
   1. [Question] <Follow-up question>
   2. [Sub-questions]
      [Sub-question 1] <First sub-question>: <Context for first sub-question>
      [Sub-question 2] <Second sub-question>: <Context for second sub-question>
      [Sub-question 3] <Third sub-question>: <Context for third sub-question>.
   ### Source
   <Source> or None (e.g., [Table 101]. DO NOT use this format: [1].).

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied.

"""

# Equity and volatility indicator
extract_101_instructions = extract_instructions + """Available source:

Table 101 has explanatory variables related to equity and volatility indicator.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

# Government bond yield
extract_102_instructions = extract_instructions + """Available source:

Table 102 has explanatory variables related to government bond yield.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

# Commodities and Futures
extract_103_instructions = extract_instructions + """Available source:

Table 103 has explanatory variables related to commodities and futures.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

# Currency exchange rates
extract_104_instructions = extract_instructions + """Available source:

Table 104 has explanatory variables related to currency exchange rates.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

# Market index
extract_105_instructions = extract_instructions + """Available source:

Table 105 has explanatory variables related to market index.

Explanatory variables:
{var_dict}.

Table data:
{table_data}."""

In [292]:
# DSC101TH
def search_equity_and_volatility(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_101_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_101,
        table_data=table_101_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

# DSC102TH
def search_government_bond_yield(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_102_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_102,
        table_data=table_102_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

# DSC103TH
def search_commodities_and_futures(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_103_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_103,
        table_data=table_103_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

# DSC104TH
def search_currency_exchange_rates(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_104_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_104,
        table_data=table_104_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

# DSC105TH
def search_market_index(state: InterviewState):
    """ Extract context from table """

    # Get state
    messages = state['messages']
    
    # LLM
    system_message = extract_105_instructions.format(
        asset_names=asset_names,
        var_dict=var_dict_105,
        table_data=table_105_data
        )
    # structured_llm = llm.with_structured_output(SearchQuery)
    extract_result = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    return {"context": [extract_result.content]}

In [293]:
answer_instructions = """You are an expert being interviewed by an analyst.

Your goal is to answer the analyst's questions based on the provided context.

Your target assets:
{asset_names}.

Your explanation target (AI model result):
{AI_model_result}.

Analyst's persona:
{persona}.

Your task:

1. Analyze the full conversation and pay particular attention to the analyst's last message, which contains sub-questions.
2. Answer the questions (`## Answer` header).
   Follow these guidelines:
    - Answer all sub-questions posed by the analyst at last turn, one by one, using the provided context.
     (Caution: DO NOT handle just single sub-quesiton.)
    - Only use the provided context. When the provided context is insufficient, DO NOT make up any answer and just mention specifically what additional context you need.
     (Caution: DO NOT leverage external or unknown contexts that are not provided.)
    - Include source next to any relevant statements (e.g., for Table 102 use [Table 102].).
    - Create a list of sources (`### Sources` header).
    
Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:

   ## Answer
   [Sub-question 1] <First sub-question>: <Answer for first sub-question>
   [Sub-question 2] <Second sub-question>: <Answer for second sub-question>
   [Sub-question 3] <Third sub-question>: <Answer for third sub-question>
   ### Sources
   <Source 1>, <Source 2>, etc. (e.g., [Table 101], [Table 102]. DO NOT use this format: [1], [2].).

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied.

Provided context:
{context}"""

In [294]:
def generate_answer(state: InterviewState):
    """ Node to answer a question """

    # Get state
    analyst = state["analyst"]
    context = state["context"]
    messages = state["messages"]

    # LLM
    system_message = answer_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result,
        persona=analyst.persona,
        context=context
        )
    answer = llm.invoke(
        [SystemMessage(content=system_message)] +
        messages
        )
    
    # Name the message as coming from the expert
    answer.name = "expert"
    
    return {"messages": [answer]}

def save_interview(state: InterviewState):
    """ Save interviews """

    # Get state
    messages = state["messages"]
    
    # Convert interview to a string
    interview = get_buffer_string(messages)
    
    return {"interview": interview}

def route_messages(state: InterviewState, 
                   name: str = "expert"):
    """ Route between question and answer """
    
    # Get state
    messages = state["messages"]
    max_num_turns = state["max_num_turns"]

    # Check the number of expert answers 
    num_responses = len(
        [m for m in messages if isinstance(m, AIMessage) and m.name == name]
    )

    # End if expert has answered more than the max turns
    if num_responses >= max_num_turns:
        return 'save_interview'

    # This router is run after each question - answer pair 
    # Get the last question asked to check if it signals the end of discussion
    last_question = messages[-2]
    
    if "I'd like to finish my interview with expert since I'm ready for impressive explanation!" in last_question.content:
        return 'save_interview'
    
    return "ask_question"

#### C. Section 작성

In [295]:
section_writer_instructions = """You are a technical writer specializing in the finance field.

Your goal is to write a section based on the provided interview transcript.

Your task:

1. Review the information of interviewer:
{description}
2. Create a narrative by gathering and understanding valid sub-question and answer pairs from the interview transcript.
3. Write Body part (`## Section` header).
   Follow these guidelines:
    - DO NOT mention any interviewer and interviewee names.
    - Aim for approximately 500 words maximum and 3 paragraphs with sub-title (`### Sub-title` header). Keep the flow of the interview as close as possible.
     (Caution: DO NOT over-abbreviate original meaningful insights and figures.)
    - Aim to preserve the provided important figures and sources.
     (Caution: When making a new sentence by combining sentences with explicit source, include the new source next to this made-up sentence. It can be multiple sources like [Table 101], [Table 102].).
4. Write Sources part (`### Sources` header).
   Follow these guidelines:
    - Create a list of sources.
    - Be sure that there is no redundant sources.

Output format:

1. Include no preamble.
2. Use markdown formatting and follow this structure:
    ## Section
    ### <Sub-title 1>
    ### <Sub-title 2>
    ### <Sub-title 3>
    ### Sources 
    <Source 1>, <Source 2>, etc. (e.g., [Table 101], [Table 102]. DO NOT use this format: [1], [2].).

Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied."""

def write_section(state: InterviewState):
    """ Node to answer a question """

    # Get state
    interview = state["interview"]
    analyst = state["analyst"]
    
    # LLM (use context)
    system_message = section_writer_instructions.format(description=analyst.description)
    section = llm.invoke(
        [SystemMessage(content=system_message)] +
        [HumanMessage(content=f"Use this interview transcript to write your section: {interview}")]
        ) 
    
    # Append it to state
    return {"sections": [section.content]}

In [296]:
# Add nodes and edges 
interview_builder = StateGraph(InterviewState)
interview_builder.add_node("ask_question", generate_question)
interview_builder.add_node("search_equity_and_volatility", search_equity_and_volatility)
interview_builder.add_node("search_government_bond_yield", search_government_bond_yield)
interview_builder.add_node("search_commodities_and_futures", search_commodities_and_futures)
interview_builder.add_node("search_currency_exchange_rates", search_currency_exchange_rates)
interview_builder.add_node("search_market_index", search_market_index)
interview_builder.add_node("answer_question", generate_answer)
interview_builder.add_node("save_interview", save_interview)
interview_builder.add_node("write_section", write_section)

# Flow
interview_builder.add_edge(START, "ask_question")
interview_builder.add_edge("ask_question", "search_equity_and_volatility")
interview_builder.add_edge("ask_question", "search_government_bond_yield")
interview_builder.add_edge("ask_question", "search_commodities_and_futures")
interview_builder.add_edge("ask_question", "search_currency_exchange_rates")
interview_builder.add_edge("ask_question", "search_market_index")
interview_builder.add_edge("search_equity_and_volatility", "answer_question")
interview_builder.add_edge("search_government_bond_yield", "answer_question")
interview_builder.add_edge("search_commodities_and_futures", "answer_question")
interview_builder.add_edge("search_currency_exchange_rates", "answer_question")
interview_builder.add_edge("search_market_index", "answer_question")
interview_builder.add_conditional_edges("answer_question", route_messages, ['ask_question', 'save_interview'])
interview_builder.add_edge("save_interview", "write_section")
interview_builder.add_edge("write_section", END)

# Interview 
memory = MemorySaver()
interview_graph = interview_builder.compile(checkpointer=memory).with_config(run_name="Conduct Interviews")

### 3-3. Interview 수행 (Parallelization)

In [297]:
from langgraph.constants import Send

# Conditional edge
def initiate_all_interviews(state: ResearchGraphState):
    """ This is the "map" step where we run each interview sub-graph using Send API """    

    # Check if human feedback
    human_analyst_feedback = state.get('human_analyst_feedback')
    if human_analyst_feedback:
        # Return to create_analysts
        return "create_analysts"

    # Otherwise kick off interviews in parallel via Send() API
    else:
        messages = [HumanMessage(content="So you said you were explaining the AI model result of portfolio optimization?")]
        return [Send("conduct_interview",
                     {"analyst": analyst,
                      "max_num_turns": max_num_turns,
                      "messages": messages}) for analyst in state["analysts"]]

### 3-4. Interpretation

### 3-5. Report 작성

In [298]:
report_writer_instructions = """You are a technical writer specializing in the finance field.

Your goal is to write a technical report that explains the AI model result of portfolio optimization by integrating the provided sections.

It is extremely important to focus on on high-level themes such as market dynamics, fund flows, and potential risks.

Your target assets:
{asset_names}.

Your explanation target (AI model result):
{AI_model_result}.

Your task:

1. Analyze all the sections provided by several analysts.
2. Think carefully about insights from the provided sections.
3. Consolidate the insights to come up with impressive explanation.
4. Create a holistic narrative that reflects your persona and connects the insights together.
   This holistic narrative focuses on high-level themes such as market dynamics, fund flows, and potential risks.
5. Write Body part (`## Analysis` header).
   Follow these guidelines:
    - Assume general readers with extensive knowledge about investment.
    - Tone and manner must be technical and clear.
    - Begin by briefly summarizing 3 explanation targets (`### Target` header).
    - Emphasize the novelty of your insights.
    - DO NOT mention any imaginary names.
    - Aim for approximately 750 words maximum and 3 paragraphs with sub-title (`### Sub-title` header). Each paragraph should correspond to individual explanation target.
     (Caution: Avoid vague phrases that are not specific and have no impact. DO NOT over-abbreviate original meaningful insights and figures.)
    - Aim to preserve the provided important figures and sources.
     (Caution: When making a new sentence by combining sentences with explicit sources, include the new source next to this made-up sentence. It can be multiple sources like [Table 101], [Table 102].).
6. Write Recommendation part.
   Follow these guidelines:
    - Assume general readers with extensive knowledge about investment.
    - Tone and manner must be technical and clear.
    - DO NOT mention any imaginary names.
    - Aim for approximately 250 words maximum and 1 paragraph.
     (Caution: Avoid vague phrases that are not specific and have no impact.)
    - Use <Score>/5 format for `### Score` header (5/5: Highly recommended, 1/5: Highly not recommended).
    - Explain the reason for the score decision (`### Reason` header).
7. Write Sources part (`### Sources` header).
   Follow these guidelines:
    - Create a list of sources.
    - Be sure that there is no redundant sources.
8. Come up with an engaging and focused title (`# Title` header).
  (Caution: Avoid vague phrases that are not specific and have no impact.)
9. Provide the original English version AND Korean version report.

Output format:

1. Include no preamble.
2. For English version, use markdown formatting and follow this structure:
    # <Title>
    ## Market Analysis Result
    ### Target
    ### <Sub-title 1>
    ### <Sub-title 2>
    ### <Sub-title 3>
    ## Recommendation
    ### Score
    ### Reason
    ### Sources (e.g., [Table 101], [Table 102]. DO NOT use this format: [1], [2].).
3. 한국어 버전에 마크다운 포맷을 적용하고 다음 구조를 따르세요:
    # <제목>
    ## 시장 분석 결과
    ### 타겟
    ### <소제목 1>
    ### <소제목 2>
    ### <소제목 3>
    ## 추천
    ### 점수
    ### 근거
    ### 출처.
    
Final review:

1. Confirm that the task guidelines are all satisfied.
2. Confirm that the output format guidelines are all satisfied."""

In [299]:
def write_report(state: ResearchGraphState):
    # Full set of sections
    sections = state["sections"]
    
    # Concat all sections together
    formatted_str_sections = "\n\n".join([f"{section}" for section in sections])
    
    # LLM
    system_message = report_writer_instructions.format(
        asset_names=asset_names,
        AI_model_result=AI_model_result)    
    report = llm.invoke(
        [SystemMessage(content=system_message)] + 
        [HumanMessage(content=f"Write a report based on the provided sections. Here are the sections: {formatted_str_sections}")])
    
    return {"content": report.content}

In [300]:
# Add nodes and edges 
builder = StateGraph(ResearchGraphState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)
builder.add_node("conduct_interview", interview_builder.compile())
# builder.add_node("interpret_risk_seeking", interpret_risk_seeking)
# builder.add_node("interpret_stability_seeking", interpret_stability_seeking)
# builder.add_node("interpret_balanced", interpret_balanced)
builder.add_node("write_report", write_report)

# Logic
builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", initiate_all_interviews, ["create_analysts", "conduct_interview"])
# builder.add_edge("conduct_interview", "interpret_risk_seeking")
# builder.add_edge("conduct_interview", "interpret_stability_seeking")
# builder.add_edge("conduct_interview", "interpret_balanced")
# builder.add_edge("interpret_risk_seeking", "write_report")
# builder.add_edge("interpret_stability_seeking", "write_report")
# builder.add_edge("interpret_balanced", "write_report")
builder.add_edge("conduct_interview", "write_report")
builder.add_edge("write_report", END)

# Compile
memory = MemorySaver()
graph = builder.compile(interrupt_before=['human_feedback'], checkpointer=memory)

### 3-6. 실행

In [301]:
# Input
thread = {"configurable": {"thread_id": "1"}}

In [302]:
# Run the graph until the first interruption
for event in graph.stream({"analysts": analysts}, # {"max_analysts": max_analysts}
                          thread, 
                          stream_mode="values"):
    
    analysts = event.get('analysts', '')
    if analysts:
        for analyst in analysts:
            print(f"Name: {analyst.name}")
            print(f"Affiliation: {analyst.affiliation}")
            print(f"Role: {analyst.role}")
            print(f"Description: {analyst.description}")
            print("-" * 100) 

Name: Emma Thompson
Affiliation: Global Equities Research
Role: Equity Market Analyst
Description: Emma focuses on the dynamics of equity markets, particularly the impact of global indices such as the FTSE 100, KOSPI, and BOVESPA on portfolio optimization. She is concerned with how these indices reflect broader economic trends and investor sentiment, and how they interact with volatility indicators like the ASX All Ordinaries Gold Open.
----------------------------------------------------------------------------------------------------
Name: Carlos Mendes
Affiliation: Latin America Bond Insights
Role: Fixed Income Strategist
Description: Carlos specializes in government bond yields, with a particular focus on Brazilian and South Korean markets. He analyzes how changes in benchmark yields, such as the 10-year and 3-year Brazilian government bonds, influence investment strategies and risk assessments in portfolio optimization.
-------------------------------------------------------------

In [303]:
# Confirm we are happy
further_feedack = None
graph.update_state(
    thread,
    {"human_analyst_feedback": further_feedack},
    as_node="human_feedback"
    )

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1efbe9a2-5570-6a84-8002-9fa82dd7e063'}}

In [304]:
# Continue
for event in graph.stream(None, thread, stream_mode="updates"):
    print("--Node--")
    node_name = next(iter(event.keys()))
    print(node_name)

--Node--
conduct_interview
--Node--
conduct_interview
--Node--
conduct_interview
--Node--
conduct_interview
--Node--
conduct_interview
--Node--
write_report


In [305]:
from IPython.display import Markdown

final_state = graph.get_state(thread)
report = final_state.values.get('content')
Markdown(report)

# Portfolio Optimization Analysis: AA-ETF_EQUITY-XLU

## Market Analysis Result

### Target

The AI model's analysis of the asset group "AA-ETF_EQUITY-XLU" reveals three distinct phases in its weight adjustment within the portfolio from November 20 to December 17, 2024. Initially, there is a gradual increase in weight, followed by a significant jump, and finally, an overall trend that suggests strategic implications. These changes are influenced by various market dynamics, fund flows, and potential risks, which are crucial for understanding the portfolio's optimization strategy.

### Gradual Increase in Weight

The gradual increase in the weight of "AA-ETF_EQUITY-XLU" from November 20 to December 16, 2024, can be attributed to several key market dynamics. The "BRAZIL BOVESPA" index, with an influence score of 0.18, and the "CME-LIVE CATTLE COMP. TRc1," with a high influence score of 0.93, played significant roles in this period. These indices' positive performances likely created a favorable environment for the asset group, encouraging its increased weight in the portfolio. Additionally, the "Refinitiv Brazil Government Benchmark Bid Yield 10 Years" contributed positively with an influence score of 0.13, reflecting a calculated approach to portfolio optimization. These elements collectively supported the strategic decision to increase exposure to this asset group, indicating a response to broader market confidence and economic conditions in Brazil [Table 101], [Table 102].

### Significant Jump in Weight

The significant jump in the weight of "AA-ETF_EQUITY-XLU" from December 16 to December 17, 2024, is linked to negative correlations with certain market variables. The "CME-MINI S&P 500 INDEX TRc1" had a negative influence score of 0.61, suggesting a downturn in its performance. Similarly, the "EUREX-DJ EURO STOXX 50 TRc1" and "NYM-PALLADIUM TRc1" showed negative influence scores of 0.62 and 0.74, respectively. These negative correlations likely prompted a rapid adjustment in the portfolio, leading to the sudden increase in weight as a strategic response to mitigate potential risks and capitalize on emerging opportunities. This shift underscores the dynamic interplay between market events and investor sentiment, highlighting the importance of agility in portfolio management [Table 103].

### Strategic Implications

The overall trend and sharp rise in the weight of "AA-ETF_EQUITY-XLU" by December 17, 2024, carry significant strategic implications for portfolio management. The performance of the "Korea Stock Exchange Composite (KOSPI)" and its influence score would have been critical in assessing broader market sentiment and economic trends. Additionally, the trend in "Gold, USD FX Composite U United States Dollar Per Troy Ounce" and its influence score provide insights into investor behavior and risk appetite. Furthermore, changes in the "London Metal Exchange (LME)-Copper Grade A Cash United States Dollar Per Metric Tonne" reflect shifts in industrial demand and global economic conditions. These strategic considerations highlight the importance of dynamic portfolio adjustments in response to evolving market conditions and underscore the need for continuous monitoring and analysis of key economic indicators [Table 104], [Table 105].

## Recommendation

### Score

4/5

### Reason

The strategic increase in the weight of "AA-ETF_EQUITY-XLU" within the portfolio from November 20 to December 17, 2024, is highly recommended due to its alignment with positive market dynamics and strategic risk management. The gradual increase reflects a calculated response to favorable economic conditions, particularly in Brazil, and the significant jump indicates a proactive approach to mitigating risks associated with negative market correlations. The overall trend suggests a strategic shift towards this asset group, supported by key economic indicators and market sentiment. However, investors should remain vigilant and continuously monitor market conditions and economic indicators to ensure the portfolio's alignment with evolving market dynamics and investment goals.

### Sources

[Table 101], [Table 102], [Table 103], [Table 104], [Table 105].

# 포트폴리오 최적화 분석: AA-ETF_EQUITY-XLU

## 시장 분석 결과

### 타겟

AI 모델의 "AA-ETF_EQUITY-XLU" 자산 그룹 분석은 2024년 11월 20일부터 12월 17일까지 포트폴리오 내에서 세 가지 뚜렷한 단계의 가중치 조정을 보여줍니다. 초기에는 점진적인 가중치 증가가 있으며, 그 후에는 상당한 점프가 있고, 마지막으로 전략적 함의를 시사하는 전반적인 추세가 있습니다. 이러한 변화는 다양한 시장 역학, 자금 흐름 및 잠재적 위험에 의해 영향을 받으며, 포트폴리오 최적화 전략을 이해하는 데 중요합니다.

### 점진적인 가중치 증가

2024년 11월 20일부터 12월 16일까지 "AA-ETF_EQUITY-XLU"의 가중치가 점진적으로 증가한 것은 여러 주요 시장 역학에 기인할 수 있습니다. "BRAZIL BOVESPA" 지수는 0.18의 영향 점수를 가지고 있으며, "CME-LIVE CATTLE COMP. TRc1"은 0.93의 높은 영향 점수를 가지고 있어 이 기간 동안 중요한 역할을 했습니다. 이러한 지수의 긍정적인 성과는 이 자산 그룹에 유리한 환경을 조성하여 포트폴리오 내에서 가중치 증가를 장려했을 가능성이 큽니다. 또한, "Refinitiv Brazil Government Benchmark Bid Yield 10 Years"는 0.13의 영향 점수로 긍정적인 기여를 하여 포트폴리오 최적화에 대한 계산된 접근 방식을 반영합니다. 이러한 요소들은 집합적으로 이 자산 그룹에 대한 노출을 증가시키려는 전략적 결정을 지원하며, 브라질의 광범위한 시장 신뢰와 경제 조건에 대한 대응을 나타냅니다 [Table 101], [Table 102].

### 가중치의 급격한 점프

2024년 12월 16일부터 12월 17일까지 "AA-ETF_EQUITY-XLU"의 가중치가 급격히 증가한 것은 특정 시장 변수와의 부정적인 상관관계와 관련이 있습니다. "CME-MINI S&P 500 INDEX TRc1"은 0.61의 부정적인 영향 점수를 가지고 있어 성과가 하락했음을 시사합니다. 마찬가지로, "EUREX-DJ EURO STOXX 50 TRc1"과 "NYM-PALLADIUM TRc1"은 각각 0.62와 0.74의 부정적인 영향 점수를 보여줍니다. 이러한 부정적인 상관관계는 포트폴리오의 급속한 조정을 촉발하여 잠재적 위험을 완화하고 새로운 기회를 활용하기 위한 전략적 대응으로 가중치의 급격한 증가를 초래했을 가능성이 큽니다. 이 변화는 시장 이벤트와 투자자 심리 간의 역동적인 상호작용을 강조하며, 포트폴리오 관리에서의 민첩성의 중요성을 강조합니다 [Table 103].

### 전략적 함의

2024년 12월 17일까지 "AA-ETF_EQUITY-XLU"의 가중치의 전반적인 추세와 급격한 증가는 포트폴리오 관리에 중요한 전략적 함의를 가집니다. 이 기간 동안 "Korea Stock Exchange Composite (KOSPI)"의 성과와 그 영향 점수는 광범위한 시장 심리와 경제 동향을 평가하는 데 중요한 역할을 했을 것입니다. 또한, "Gold, USD FX Composite U United States Dollar Per Troy Ounce"의 추세와 그 영향 점수는 투자자 행동과 위험 선호도를 이해하는 데 도움을 줍니다. 더 나아가, "London Metal Exchange (LME)-Copper Grade A Cash United States Dollar Per Metric Tonne"의 변화는 산업 수요와 글로벌 경제 조건의 변화를 반영합니다. 이러한 전략적 고려 사항은 변화하는 시장 조건에 대한 동적 포트폴리오 조정의 중요성을 강조하며, 주요 경제 지표의 지속적인 모니터링과 분석의 필요성을 강조합니다 [Table 104], [Table 105].

## 추천

### 점수

4/5

### 근거

2024년 11월 20일부터 12월 17일까지 포트폴리오 내에서 "AA-ETF_EQUITY-XLU"의 가중치 증가 전략은 긍정적인 시장 역학 및 전략적 위험 관리와의 정렬로 인해 강력히 추천됩니다. 점진적인 증가는 특히 브라질의 유리한 경제 조건에 대한 계산된 대응을 반영하며, 상당한 점프는 부정적인 시장 상관관계와 관련된 위험을 완화하기 위한 적극적인 접근 방식을 나타냅니다. 전반적인 추세는 주요 경제 지표와 시장 심리에 의해 뒷받침되는 이 자산 그룹으로의 전략적 전환을 시사합니다. 그러나 투자자들은 포트폴리오가 변화하는 시장 역학 및 투자 목표와의 정렬을 보장하기 위해 시장 조건과 경제 지표를 지속적으로 모니터링해야 합니다.

### 출처

[Table 101], [Table 102], [Table 103], [Table 104], [Table 105].

In [306]:
report

'# Portfolio Optimization Analysis: AA-ETF_EQUITY-XLU\n\n## Market Analysis Result\n\n### Target\n\nThe AI model\'s analysis of the asset group "AA-ETF_EQUITY-XLU" reveals three distinct phases in its weight adjustment within the portfolio from November 20 to December 17, 2024. Initially, there is a gradual increase in weight, followed by a significant jump, and finally, an overall trend that suggests strategic implications. These changes are influenced by various market dynamics, fund flows, and potential risks, which are crucial for understanding the portfolio\'s optimization strategy.\n\n### Gradual Increase in Weight\n\nThe gradual increase in the weight of "AA-ETF_EQUITY-XLU" from November 20 to December 16, 2024, can be attributed to several key market dynamics. The "BRAZIL BOVESPA" index, with an influence score of 0.18, and the "CME-LIVE CATTLE COMP. TRc1," with a high influence score of 0.93, played significant roles in this period. These indices\' positive performances likely